In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
import google.generativeai as genai
import time
import ast

In [14]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [34]:
filtrados_com_num_excel = pd.read_excel(r"C:\Users\sergi\OneDrive\Área de Trabalho\INSPER\entidades\insper data\2025.1\aecom\repositorio\Projeto_InsperData_AECOM\testes_gemini\processos_filtrados.xlsx")

In [47]:
filtrados_com_num_excel

,Unnamed: 0,processoID,processoAnexoID,Download copia,Tribunal,num_processo
0,7499,619399554,1676545022,http://jud-anexos.digesto.com.br/d4307d87fd496...,TJAL,0000010-75.2024.8.02.0356
1,7502,619399554,1676545211,http://jud-anexos.digesto.com.br/c1541200bd444...,TJAL,0000010-75.2024.8.02.0356
2,8949,740609736,2428672443,http://jud-anexos.digesto.com.br/2b8f18d89d911...,TJPR,0000135-52.2021.8.16.0067
3,8952,740609736,2428672467,http://jud-anexos.digesto.com.br/f97ce1990319d...,TJPR,0000135-52.2021.8.16.0067
4,8956,740609736,2428672520,http://jud-anexos.digesto.com.br/3021c9e81bbd3...,TJPR,0000135-52.2021.8.16.0067
...,...,...,...,...,...,...
4293,13855,494410810,257604195,http://jud-anexos.digesto.com.br/550b4f243c10a...,TRF1,Não encontrado
4294,13856,494410810,257604196,http://jud-anexos.digesto.com.br/ec272e1878687...,TRF1,Não encontrado
4295,13857,494410810,257604197,http://jud-anexos.digesto.com.br/b645d0f74e2e9...,TRF1,Não encontrado
4296,13858,494410810,257604198,http://jud-anexos.digesto.com.br/124da1a65a0e3...,TRF1,Não encontrado


In [35]:
filtrados_true = pd.read_excel('respostas_danoambiental_50.xlsx')

In [36]:
filtrados_com_num = filtrados_com_num_excel
processos_unicos = list(filtrados_com_num['num_processo'].unique())
display(filtrados_com_num.loc[filtrados_com_num['num_processo']=='Não encontrado'])
processos_unicos.remove('Não encontrado')

,Unnamed: 0,processoID,processoAnexoID,Download copia,Tribunal,num_processo
4131,169,655333385,1942855138,http://jud-anexos.digesto.com.br/30f8d4e93b94a...,TJBA,Não encontrado
4132,173,655333385,1942855153,http://jud-anexos.digesto.com.br/f2d578e94d700...,TJBA,Não encontrado
4133,194,655333385,1942855177,http://jud-anexos.digesto.com.br/ac2380dd6d952...,TJBA,Não encontrado
4134,195,655333385,1942855178,http://jud-anexos.digesto.com.br/042db8c607606...,TJBA,Não encontrado
4135,196,655333385,1942855179,http://jud-anexos.digesto.com.br/a34562b835d2a...,TJBA,Não encontrado
...,...,...,...,...,...,...
4293,13855,494410810,257604195,http://jud-anexos.digesto.com.br/550b4f243c10a...,TRF1,Não encontrado
4294,13856,494410810,257604196,http://jud-anexos.digesto.com.br/ec272e1878687...,TRF1,Não encontrado
4295,13857,494410810,257604197,http://jud-anexos.digesto.com.br/b645d0f74e2e9...,TRF1,Não encontrado
4296,13858,494410810,257604198,http://jud-anexos.digesto.com.br/124da1a65a0e3...,TRF1,Não encontrado


In [37]:
testes = filtrados_true[1:100]

In [38]:
processo_anexo_id_lista = testes['processoAnexoID'].tolist()

In [48]:
processo_anexo_id_lista

[2908924070,
 1419499656,
 657568057,
 1088560478,
 1186043580,
 1397328389,
 1299854564,
 1462784031,
 1484217656,
 2193427480,
 2924793560,
 374401292,
 2002955426,
 2583016248,
 2447744989,
 2890794934,
 2374549812,
 2451590480,
 2090274323,
 2229235049,
 2332883980,
 2887865420,
 2582226524,
 1249838163,
 1987957126,
 2431711293,
 1124178884,
 2722162734,
 1053877654,
 1148584788,
 2124556719,
 2589045540,
 2762584664,
 1182706696,
 2925804315,
 2924963112,
 2635090005,
 2718369649,
 2935865691,
 2075508189,
 1494896928,
 2015181677,
 2014041729,
 812664082,
 1923392718,
 2271997843,
 2685702174,
 2762585026,
 2447098711]

In [40]:
def analise(texto_extraido,respostas,numero_processo,processoAnexoID,link):

    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial. Seu objetivo é analisar esse texto e identificar se ele trata de um caso de dano ambiental e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
     Se não houver, retorne: "Não encontrado georreferencia"

    [1] Unidade Federativa (UF) completa (ex: "Maranhão", "São Paulo" etc.)
     Se não houver, retorne: "Não encontrado UF"

    [2] Município ou cidade do local afetado.
     Se não houver, retorne: "Não encontrado cidade"

    [3] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
     Se não houver, retorne: "Não encontrado responsavel"

    [4] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
     Se não houver, retorne: "Não encontrado pessoa"

    [5] Data do impacto ambiental no formato: DD/MM/AA
     Se não houver, retorne: "Não encontrado data"

    [6] Extensão da área afetada (ex: "120 hectares", "15.000 m²", etc.)
     Se não houver, retorne: "Não encontrado área"

    [7] Unidade de medida da área (ex: "hectares", "m²", etc.)
     Se não houver, retorne: "Não encontrado medida"

    [8] Metodologia de punição aplicada (ex: auto de infração, processo administrativo, etc.)
     Se não houver, retorne: "Não encontrado método"

    [9] Compensação não monetária atribuída  (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
     Se não houver, retorne: "Não encontrada compensação"

    [10] Categoria da compensação: "Multas Administrativas", "Compensações Financeiras", "Obrigações de Fazer (com custo)", " Custas Judiciais e Acordos" ou "Valoração Econômica".
     Se não houver, retorne: "Não encontrada categoria"

    [11] Tipo de Valor da compensção de acordo com a categoria da compensação: para multas administrativas existe 'Multa simples', 'Multa diária',entre outras. para compensações financeiras existe 'Indenização por dano material', 'Indenização por dano moral coletivo','Compensação ambiental (Lei do SNUC)' entre outros. Para obrigações de fazer (com custo) existe por exemplo 'Reflorestamento ou recomposição vegetal', 'Descontaminação de solo e água', entre outros. Para C'ustas Judiciais e Acordos'	existe 'Custas processuais e honorários' por exemplo. Para 'Valoração Econômica' existe	'Valor de mercado dos bens naturais afetados' por exemplo.
     Se não houver, retorne: "Não encontrado tipo valor"

    [12] Forma de Cálculo ou valor (por exemplo: R$ 5.000, R$ 500 a R$ 10 mil/dia., reflorestamento a R$ 30 mil/hectare, R$ 3 milhões para limpar lençol freático, 10% a 20% do valor da causa)

    IMPORTANTE: Não retorne o texto original, Não inclua explicações ou formatações extras, Apenas a lista com os 13 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 13 elementos (de índice 0 a 12), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,numero_processo)
    lista = [x for x in lista if x != '']
    lista.append(processoAnexoID)
    lista.append(link)
    
    respostas[numero_processo] = lista
    return respostas
    

In [45]:
respostas = {}

for processo in filtrados_true: 
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():

        processoAnexoID = row['processoAnexoID']
        print(processoAnexoID)

        if processoAnexoID == '1987957126': 
            print('entrou aqui')
            print(processoAnexoID)
            link = row['Download copia']
            response = requests.get(link)
            response.encoding = 'utf-8'
            content_type = response.headers.get('Content-Type', '')
            
            if 'pdf' in content_type:
                with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                    tmp_file.write(response.content)
                    tmp_path = tmp_file.name

                with pymupdf.open(tmp_path) as doc:
                    for page in doc:
                        texto += page.get_text()
            
            elif 'html' in content_type:
                soup = BeautifulSoup(response.content, 'html.parser')
                texto += soup.get_text()
            
            # Agora está DENTRO do IF
            analise(texto,respostas,processo,processoAnexoID,link)


In [46]:
respostas  

{}

In [ ]:
respostas_teste2 = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste2.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria',
    'processoAnexoID',
    'link'
    ]